## CAPSTONE PROJECT ASSIGNMENT

### Applied Data Science Capstone Project by Zhansaya Zhapar

## Table of contents

* [Data Description](#data)
    * [New York Neighbourhoods Data Preparation](#newyork_data)
    * [Foursquare API Data Preparation](#foursquare)
    * [Airbnb Data Preparation](#airbnb)

## Data Description <a name="data"></a>
To fulfill the aim of my project I need to gather data that contain:

* location of a specific apartment in NY being offered in Airbnb
* the list and amount of outdoor and recreation venues in the neighborhood of that apartment
* the list and amount of transport venues in the neighborhood of that apartment
* the list and amount of grocery shops in the neighborhood of that apartment
* number of reviews made by users that stayed in that apartment 
* price of that apartment


Aforementioned data will be extracted and then processed from the following sources:

1. **Foursquare API:**

* all the required information about nearby venues in the neighborhood of that apartment 

2. **Airbnb open source data:** 

* price, number of reviews


Also, I will use geopy library to get the latitude and longitude values of New York City.



#### Data Preparation





Let's import all the required libraries:

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('seaborn')
import seaborn as sns
import folium

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


### New York Neighbourhoods Data Preparation  <a name="newyork_data"></a>

Now, let's download New York city data and load json file:

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Let's see, how our data looks like:

In [3]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Let's extract only the data that we need. As we see, it's stored in the "features":

In [4]:
neighbourhoods_data = newyork_data['features']

In [5]:
column_names = ['Borough', 'neighbourhood', 'latitude', 'longitude'] #initiate new dataframe's columns
neighbourhoods = pd.DataFrame(columns=column_names) #inititate the dataframe with these columns

In [6]:
#Function that will extract and write the data into our new dataframe:
for data in neighbourhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighbourhood_name = data['properties']['name']
        
    neighbourhood_latlon = data['geometry']['coordinates']
    neighbourhood_lat = neighbourhood_latlon[1]
    neighbourhood_lon = neighbourhood_latlon[0]
    
    neighbourhoods = neighbourhoods.append({'Borough': borough,
                                          'neighbourhood': neighbourhood_name,
                                          'latitude': neighbourhood_lat,
                                          'longitude': neighbourhood_lon}, ignore_index=True)

Let's see the first 5 lines of the dataframe that contains borough, neighbourhood and its coordinates information:

In [7]:
neighbourhoods.head()

,Borough,neighbourhood,latitude,longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Since for this project I have decided to focus on Brooklyn area, let's create new dataframe that will contain only the neighbourhoods in Brooklyn:

In [8]:
brooklyn_neighbourhoods = neighbourhoods[neighbourhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
print(brooklyn_neighbourhoods.shape)
brooklyn_neighbourhoods.head()

(70, 4)


,Borough,neighbourhood,latitude,longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


## Foursquare API Data Preparation <a name="foursquare"></a>

Now, in order to use FOURSQUARE API I am going to define credentials info:

In [9]:
CLIENT_ID = '5PJJA0IAXCNAXBXOYYZOWRQFU3KRLC23F4BPSXDF131LCKGF' # my Foursquare ID
CLIENT_SECRET = 'ELKIDEW2JAYKRZS4LWDJ0WRMMVJNLG4QY5HDF0TBEJSJSGZK' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: 5PJJA0IAXCNAXBXOYYZOWRQFU3KRLC23F4BPSXDF131LCKGF
CLIENT_SECRET:ELKIDEW2JAYKRZS4LWDJ0WRMMVJNLG4QY5HDF0TBEJSJSGZK


Let's find how many outdoor venues are present in each neighbourhood:

Before making a request I have searched and found the id of outdoor and recreation venues, which is:
**4d4b7105d754a06377d81259**

In [10]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
def getNearbyOutdoorVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06377d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighbourhood', 
                 'neighbourhood Latitude', 
                 'neighbourhood Longitude', 
                 'Venue', 
                'Venue Latitude', 
                 'Venue Longitude', 
                 'Venue Category']
    
    return(nearby_venues)

In [11]:
# Brooklyn outdoor & recreation venues:
brooklyn_outdoor_venues = getNearbyOutdoorVenues(brooklyn_neighbourhoods['neighbourhood'],
                                   latitudes=brooklyn_neighbourhoods['latitude'],
                                   longitudes=brooklyn_neighbourhoods['longitude']
                                  )

Let's see what has been returned from our request:

In [12]:
brooklyn_outdoor_venues.head()

,neighbourhood,neighbourhood Latitude,neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Funtime USA,40.626297,-74.029701,Playground
1,Bay Ridge,40.625801,-74.030621,Dahn Yoga,40.626273,-74.030208,Yoga Studio
2,Bay Ridge,40.625801,-74.030621,Monster Baseball Field,40.630378,-74.028043,Baseball Field
3,Bay Ridge,40.625801,-74.030621,Beach Bootay,40.623909,-74.033012,Vineyard
4,Bay Ridge,40.625801,-74.030621,Bay Ridge Crossfit,40.624143,-74.030823,Gym / Fitness Center


We can see from the above in the categories that there are many venues like playgrounds, yoga studios and fitness centers in each neighbourhood. But we need to know differences between neighbourhoods and know exact number of interested venues:

In [13]:
# how many venues returned for each neighbourhood:
number_outdoor = brooklyn_outdoor_venues.groupby('neighbourhood').count()

In [14]:
number_outdoor = number_outdoor.rename(columns ={"neighbourhood Latitude": "number_of_outdoors"})
#Let's drop the columns that we not intereste in:
number_outdoor = number_outdoor.drop(['neighbourhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'], axis = 1) 
number_outdoor.head()

,number_of_outdoors
neighbourhood,
Bath Beach,25
Bay Ridge,30
Bedford Stuyvesant,24
Bensonhurst,14
Bergen Beach,13


Let's find how many grocery stores are present in each neighbourhood:
(Id of this category is 4bf58dd8d48988d1f9941735)

In [15]:
def getNearbyFoodShops(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d1f9941735&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighbourhood', 
                 'neighbourhood Latitude', 
                 'neighbourhood Longitude', 
                 'Venue', 
                'Venue Latitude', 
                 'Venue Longitude', 
                 'Venue Category']
    
    return(nearby_venues)

In [16]:
brooklyn_food_shops = getNearbyFoodShops(brooklyn_neighbourhoods['neighbourhood'],
                                   latitudes=brooklyn_neighbourhoods['latitude'],
                                   longitudes=brooklyn_neighbourhoods['longitude']
                                  )

In [17]:
brooklyn_food_shops.head()

,neighbourhood,neighbourhood Latitude,neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Brooklyn Market,40.626939,-74.029948,Grocery Store
1,Bay Ridge,40.625801,-74.030621,Kings Cellars,40.625589,-74.030462,Liquor Store
2,Bay Ridge,40.625801,-74.030621,Associated Supermarket,40.628072,-74.029176,Grocery Store
3,Bay Ridge,40.625801,-74.030621,Appletree natural foods,40.628165,-74.028864,Farmers Market
4,Bay Ridge,40.625801,-74.030621,Met Foodmarkets,40.625382,-74.026549,Grocery Store


To extract the number of grocery & food shops nearby, let's create dataframe with numbers only:

In [18]:
number_food_shops = brooklyn_food_shops.groupby('neighbourhood').count()
number_food_shops = number_food_shops.rename(columns ={"neighbourhood Latitude": "number_of_food_shops"})
number_food_shops = number_food_shops.drop(['neighbourhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'], axis = 1) 
number_food_shops.head()

,number_of_food_shops
neighbourhood,
Bath Beach,14
Bay Ridge,25
Bedford Stuyvesant,27
Bensonhurst,12
Boerum Hill,31


Now, let's explore travel and transport spots in each neighbourhood:
(id of this venue category is 4d4b7105d754a06379d81259)

In [19]:
def getNearbyTransport(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06379d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighbourhood', 
                 'neighbourhood Latitude', 
                 'neighbourhood Longitude', 
                 'Venue', 
                'Venue Latitude', 
                 'Venue Longitude', 
                 'Venue Category']
    
    return(nearby_venues)

In [20]:
brooklyn_transport = getNearbyTransport(brooklyn_neighbourhoods['neighbourhood'],
                                   latitudes=brooklyn_neighbourhoods['latitude'],
                                   longitudes=brooklyn_neighbourhoods['longitude']
                                  )
brooklyn_transport.head()

,neighbourhood,neighbourhood Latitude,neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,MTA Bus - 3 Av & 86 St (B16/B37),40.624007,-74.033274,Bus Stop
1,Bay Ridge,40.625801,-74.030621,MTA Subway - 86th St (R),40.622812,-74.028383,Metro Station
2,Bay Ridge,40.625801,-74.030621,Enterprise Rent-A-Car,40.621121,-74.028314,Rental Car Location
3,Bay Ridge,40.625801,-74.030621,3rd Ave,40.624310,-74.024782,Moving Target
4,Bay Ridge,40.625801,-74.030621,Best Western Gregory Hotel,40.624357,-74.027360,Hotel


Repeating the process of extracting the numbers for each neighburhood:

In [21]:
number_transport = brooklyn_transport.groupby('neighbourhood').count()
number_transport = number_transport.rename(columns ={"neighbourhood Latitude": "number_of_transportation_spots"})
number_transport = number_transport.drop(['neighbourhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'], axis = 1) 
number_transport.head()

,number_of_transportation_spots
neighbourhood,
Bath Beach,21
Bay Ridge,24
Bedford Stuyvesant,41
Bensonhurst,7
Bergen Beach,1


Now, let's combine our data provided by Foursquare API:

In [22]:
merged_venues = pd.merge(number_transport, number_food_shops, on='neighbourhood')
merged_venues = pd.merge(merged_venues, number_outdoor, on='neighbourhood')
merged_venues.head()

,number_of_transportation_spots,number_of_food_shops,number_of_outdoors
neighbourhood,,,
Bath Beach,21,14,25
Bay Ridge,24,25,30
Bedford Stuyvesant,41,27,24
Bensonhurst,7,12,14
Boerum Hill,48,31,50


Let's see how many neighbourhoods our dataframe has:

In [23]:
merged_venues.shape

(66, 3)

### Airbnb Data Preparation <a name="airbnb"></a>

I downloaded listing.csv file with airbnb data from the following website:
http://insideairbnb.com/get-the-data.html

In [24]:
df_raw = pd.read_csv('listings.csv')
df_raw.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
1,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,279,2019-08-29,4.62,1,192
2,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
3,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,75,2019-07-21,0.59,1,13
4,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,45,49,2017-10-05,0.39,1,0


Let's drop unneccessary columns:

In [25]:
df = df_raw.drop(['name','host_id', 'minimum_nights', 'last_review', 'reviews_per_month', 'calculated_host_listings_count'], axis = 1) 
df.head()

,id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,availability_365
0,3647,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,0,365
1,3831,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,279,192
2,5022,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,9,0
3,5099,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,75,13
4,5121,Garon,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,49,0


In [26]:
df.shape

(48377, 10)

Now, it is important to identify missing values and decide how to handle them:

In [27]:
df_null = df[df.isna().any(axis=1)]
df_null

,id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,availability_365
224,60680,NaN,Manhattan,East Village,40.72654,-73.98049,Entire home/apt,99,134,313
773,289703,NaN,Manhattan,SoHo,40.72530,-73.99916,Entire home/apt,249,8,0
5426,4140622,NaN,Brooklyn,Bedford-Stuyvesant,40.68514,-73.93207,Entire home/apt,100,0,0
5807,4446862,NaN,Brooklyn,Crown Heights,40.67512,-73.96146,Private room,50,0,0
6286,4763327,NaN,Brooklyn,Greenpoint,40.72035,-73.95355,Entire home/apt,195,1,0
7816,6304803,NaN,Brooklyn,Clinton Hill,40.69322,-73.96554,Entire home/apt,300,1,0
7893,6360224,NaN,Brooklyn,Bushwick,40.70146,-73.92792,Private room,37,1,0
8731,6992973,NaN,Brooklyn,Williamsburg,40.71838,-73.95630,Entire home/apt,145,0,0
9387,7556587,NaN,Manhattan,Harlem,40.82929,-73.94182,Private room,28,1,0
14144,11646497,NaN,Brooklyn,Bedford-Stuyvesant,40.68486,-73.93200,Private room,89,0,0


So, it has been found that only host names are missing. 
I will replace NaN by "Not specified" string value:

In [28]:
df = df.replace(np.nan, 'Not specified', regex=True)
#check loc 224 that name changed:
df.loc[224]

id                               60680
host_name                Not specified
neighbourhood_group          Manhattan
neighbourhood             East Village
latitude                       40.7265
longitude                     -73.9805
room_type              Entire home/apt
price                               99
number_of_reviews                  134
availability_365                   313
Name: 224, dtype: object

Now, let's double-check that we don't have any missing values:

In [29]:
print(df.isnull().sum().sum())

0


In [30]:
df.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48377 entries, 0 to 48376
Data columns (total 10 columns):
id                     48377 non-null int64
host_name              48377 non-null object
neighbourhood_group    48377 non-null object
neighbourhood          48377 non-null object
latitude               48377 non-null float64
longitude              48377 non-null float64
room_type              48377 non-null object
price                  48377 non-null int64
number_of_reviews      48377 non-null int64
availability_365       48377 non-null int64
dtypes: float64(2), int64(4), object(4)
memory usage: 3.7+ MB


Also, to clean the data and work with the relevant info, let's drop the listings that offer their apartment less than 100 days in a year. 

**Assumption 1:** I assume that the client would want to stay in apartment for 3 months in average.
So, let's first see, how many apartments would satisfy this assumption?

In [31]:
df_av = df.loc[df['availability_365'] > 100]
df_av.shape

(19272, 10)

In [32]:
#let's drop the listings with less than 100 days availability:
df = df[df['availability_365'] >= 100].reset_index(drop = True)
print(df.shape)
df.head()

(19323, 10)


,id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,availability_365
0,3647,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,0,365
1,3831,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,279,192
2,5178,Shunichi,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,443,246
3,5222,Marilyn,Manhattan,East Village,40.72764,-73.97949,Entire home/apt,116,94,347
4,5803,Laurie,Brooklyn,South Slope,40.66829,-73.98779,Private room,89,171,332


**Assumption 2:** The client wants to stay in separate entire apartment (not in a shared one):

In [33]:
df = df[df['room_type'] == 'Entire home/apt'].reset_index(drop = True)
print(df.shape)
df.head()

(10227, 10)


,id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,availability_365
0,3831,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,279,192
1,5222,Marilyn,Manhattan,East Village,40.72764,-73.97949,Entire home/apt,116,94,347
2,6848,Allen & Irina,Brooklyn,Williamsburg,40.70837,-73.95352,Entire home/apt,140,160,111
3,7097,Jane,Brooklyn,Fort Greene,40.69169,-73.97185,Entire home/apt,199,203,311
4,7750,Sing,Manhattan,East Harlem,40.79685,-73.94872,Entire home/apt,190,0,302


Now, we can delete columns with room type and availability from our dataframe:

In [34]:
df = df.drop(['room_type', 'availability_365'], axis = 1) 
df.head()

,id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,price,number_of_reviews
0,3831,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,89,279
1,5222,Marilyn,Manhattan,East Village,40.72764,-73.97949,116,94
2,6848,Allen & Irina,Brooklyn,Williamsburg,40.70837,-73.95352,140,160
3,7097,Jane,Brooklyn,Fort Greene,40.69169,-73.97185,199,203
4,7750,Sing,Manhattan,East Harlem,40.79685,-73.94872,190,0


In [35]:
#What is the shape of our dataframe now:
df.shape

(10227, 8)

Let's see the unique boroughs, i.e. neighbourhood groups in the datframe:

In [36]:
df['neighbourhood_group'].unique()

array(['Brooklyn', 'Manhattan', 'Queens', 'Bronx', 'Staten Island'],
      dtype=object)

Keeping in mind that for this particular project I am interested only in Brooklyn area, let's extract listing located in Brooklyn only:

In [37]:
df_brooklyn = df[df['neighbourhood_group'] == 'Brooklyn'].reset_index(drop=True)
print(df_brooklyn.shape)
df_brooklyn.head()

(3429, 8)


,id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,price,number_of_reviews
0,3831,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,89,279
1,6848,Allen & Irina,Brooklyn,Williamsburg,40.70837,-73.95352,140,160
2,7097,Jane,Brooklyn,Fort Greene,40.69169,-73.97185,199,203
3,8490,Nathalie,Brooklyn,Bedford-Stuyvesant,40.68371,-73.94028,120,96
4,12299,Abdul,Brooklyn,South Slope,40.66278,-73.97966,120,15


Let's see, how many in how many neighbourhoods our left listings are located?

In [38]:
df_brooklyn['neighbourhood'].unique().shape

(48,)

Interesting that from Foursquare API, we have obtained nearby venues for 66 neighbourhoods of Brooklyn.

But only 48 out of them have apartments for renting by airbnb that satisfy our assumptions.

Now, I would like to group airbnb dataframe by neighbourhood and see the average price and the number of reviews for each:

In [39]:
df_airbnb = df_brooklyn.groupby('neighbourhood').mean()
df_airbnb = df_airbnb.drop(['id', 'latitude', 'longitude'], axis = 1) 
df_airbnb.head()

,price,number_of_reviews
neighbourhood,,
Bath Beach,132.800000,49.200000
Bay Ridge,205.542857,47.771429
Bedford-Stuyvesant,172.110818,61.992084
Bensonhurst,112.150000,57.400000
Bergen Beach,104.500000,23.125000


#### Merging all data in final dataframe:

In [40]:
df_final = pd.merge(df_airbnb, merged_venues, on='neighbourhood')

df_final.head()

,price,number_of_reviews,number_of_transportation_spots,number_of_food_shops,number_of_outdoors
neighbourhood,,,,,
Bath Beach,132.800000,49.200000,21,14,25
Bay Ridge,205.542857,47.771429,24,25,30
Bensonhurst,112.150000,57.400000,7,12,14
Boerum Hill,225.282051,40.897436,48,31,50
Borough Park,146.166667,31.666667,22,16,2


In [41]:
df_final.shape

(38, 5)